# Projet Réseaux euclidiens en cryptographie

## Première partie : Implémentation du système cryptographique

In [1]:
"""
Génère une matrice unimodulaire aléatoire.

Paramètres :
- n (int) : Taille de la matrice carrée (n x n).

Retour :
- Matrix : Une matrice unimodulaire de dimensions n x n (déterminant égal à ±1).
"""
def generate_unimodular_matrix(n):
    U = Matrix(ZZ, n,n)
    
    while not (U.det() in [-1, 1]):
        U = identity_matrix(ZZ, n,n)
        ind = randint(0,n-1)
        
        if randint(0,1): # Modifie une ligne ou une colonne avec 50% de chances
            U[ind] = [randint(-1,1) for i in range(n)]
            U[ind, ind] = 1
        else:
            for i in range(n):
                U[i, ind] = randint(-1,1)
            U[ind, ind] = 1
    return U

In [2]:
"""
Génère une clé publique et une clé privée pour un chiffrement basé sur des matrices.

Paramètres :
- n (int) : Taille des matrices carrées (n x n).
- l (int) : Amplitude des coefficients dans la matrice privée (valeur par défaut : 4).
- debug (bool) : Si True, affiche les matrices générées pour le débogage.

Retour :
- Bpriv (Matrix) : Matrice privée.
- Bpub (Matrix) : Matrice publique.
- U (Matrix) : Matrice unimodulaire utilisée pour transformer Bpriv en Bpub.
"""
def KeyGen(n, l = 4, debug = False):
    Bpriv = MatrixSpace(ZZ, n,n).random_element(x = -l, y = l + 1 , distribution = 'uniform')
    while Bpriv.det() == 0:
        Bpriv = MatrixSpace(ZZ, n,n).random_element(x = -l, y = l + 1 , distribution = 'uniform')
    Bpriv = Bpriv.LLL()
        
    U = generate_unimodular_matrix(n)
        
    Bpub = U * Bpriv
    
    if(debug):
        print("Matrice B (clé privée) :")
        print(Bpriv)
        print("\nMatrice U (unimodulaire) :")
        print(U)
        print("\nMatrice B' (clé publique) :")
        print(Bpub)

    return Bpriv, Bpub, U

In [3]:
"""
Chiffre un message à l'aide de la clé publique.

Paramètres :
- Bpub (Matrix) : Clé publique utilisée pour le chiffrement.
- m (vector) : Message à chiffrer, sous la forme d'un vecteur.
- breakPoint (int) : Nombre maximal d'itérations pour ajuster l'erreur (valeur par défaut : 128).

Retour :
- c (vector) : Message chiffré
"""
def Chiffrement(Bpub, m, breakPoint = 128):
    n = Bpub.nrows()
    r = round(min([Bpub.column(i).norm(2) for i in range(n)]) / 6)
    err = vector(ZZ, [randint(0, 1) for _ in range(n)])
    boucle = 0
    while err.norm(2) >= r:
        boucle += 1
        err = vector(ZZ, [randint(-1, 1) for _ in range(n)])
        if(boucle > breakPoint):
            err = vector(ZZ, [randint(0,1) if(i < r) else 0 for i in range(n)])
            break
    return m*Bpub + err

"""
Déchiffre un message à l'aide de la clé privée.

Paramètres :
- Bpriv (Matrix) : Matrice privée pour le déchiffrement.
- U (Matrix) : Matrice unimodulaire utilisée dans la génération des clés.
- c (vector) : Message chiffré.

Retour :
- m (vector) : Message clair, sous la forme d'un vecteur.
"""
def Dechiffrement(Bpriv, U, c):
    m_prim = c*Bpriv.inverse()
    m_sec = m_prim.apply_map(lambda x: round(x))
    m = m_sec * U.inverse()

    return m

In [4]:
def Dechiffrement(Bpriv, U, c):
    m_prim = c*Bpriv.inverse()
    m_sec = m_prim.apply_map(lambda x: math.ceil(x) if x % 1 == 0.5 else round(x))
    m = m_sec * U.inverse()

    return m

In [5]:
# Test des fonctions de génération de clé, de chiffrement et déchiffrement
n = 200
good = 0
nb = 5
ERR = []
nberr = 0

for i in range(nb):

    Bpriv, Bpub, U = KeyGen(n)

    m = vector(ZZ, [randint(0, 1) for _ in range(n)])
    c = Chiffrement(Bpub, m)

    m2 = Dechiffrement(Bpriv, U, c)
    if(m == m2):
        good += 1
    else:
        nberr += 1
        e = m - m2
        print(e)
        ERR.append(sum([0 if (e[i] == 0) else 1 for i in range(n)]))
print(f"Le taux de réussite dans le déchiffrement pour des matrices de tailles {n} est de {good / nb * 100}%")
if(good / nb * 100 < 100):
    print(f"Le taux de bits incorrectement déchiffré en cas d'erreur est de {round(sum(ERR) / nberr / n * 100)}%")

(0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
Le taux de réussite dans le déchiffrement pour des matrices de tailles 200 est de 80%
Le taux de bits incorrectement déchiffré en cas d'erreur est de 1%


## Cryptanalyse 

### Attaque par force brute

In [6]:
from itertools import product

"""
Génère tous les vecteurs possibles de dimension donnée avec des coefficients
dans {0, -1, 1} et une norme inférieure à une borne.

Paramètres :
- dim (int) : Dimension des vecteurs à générer.
- r (int) : Borne supérieure pour la norme Euclidienne.

Retour :
- (iterator) : Itérateur sur les vecteurs respectant les contraintes.
"""
def generate_vectors_iterative(dim, r):
    for v in product([0, -1, 1], repeat=dim):
        v = vector(v)
        if v.norm() < r:
            yield v

"""
Effectue une attaque par force brute pour déchiffrer un message.

Paramètres :
- Bpub (Matrix) : Matrice publique utilisée pour le chiffrement.
- c (vector) : Message chiffré.
- n (int) : Dimension du message clair et de l'erreur.

Retour :
- candidates (list) : Liste des messages candidats sous forme de vecteurs.
"""
def brute_force_attack(Bpub, c, n):
    r = round(min([Bpub.column(i).norm(2) for i in range(n)]) / 3)
    Bpub_inv = Bpub.inverse()
    candidates = []
    
    c_prim = c * Bpub_inv
    for e in generate_vectors_iterative(n, r):
        possible_m = c_prim - Bpub_inv * e
        possible_m = possible_m.apply_map(lambda x: round(x))

        if all(x in [0, 1] for x in possible_m):
            candidates.append(possible_m)

            if possible_m == m:
                print("Message trouvé:", possible_m)
                return candidates

    print("Message non trouvé, augmentation de la norme maximale.")
    return candidates

In [7]:
n = 32
Bpriv, Bpub, U = KeyGen(n)

m = vector(ZZ, [randint(0, 1) for _ in range(n)])
print("Message recherché:",m)
c = Chiffrement(Bpub, m)
print("Parmi les candidats suivants :", brute_force_attack(Bpub, c,  n))

Message recherché: (1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0)
Message trouvé: (1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0)
Parmi les candidats suivants : [(1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0)]


In [9]:
def nearest_plane_attack(c, Bpub, n):
    # Appliquer la réduction LLL pour obtenir une base réduite
    Bpub_lll = Bpub.LLL()
    
    # Calculer l'inverse de la base publique pour obtenir Bpub_inv
    Bpub_inv = Bpub_lll.inverse()

    # Représenter c comme une combinaison linéaire des vecteurs de la base
    c_prim = Bpub_inv * c

    # Appliquer l'algorithme Nearest-plane itératif
    basis = Bpub_inv.transpose()
    
    # Coefficients de la représentation de c en fonction de la base
    coefficients = c_prim.list()

    # Initialisation de la base et du vecteur c
    Bnew = Bpub_lll
    c_new = c

    # Iteration de l'algorithme Nearest-plane sur les coefficients
    for i in range(n-1, -1, -1):
        coeff = coefficients[i]
        rounded_coeff = round(coeff)
        c_new = c_new - rounded_coeff * Bnew[i]

        # Mise à jour de la base
        Bnew = matrix(list(Bnew)[:i])

    # Calculer le vecteur d'erreur et afficher le message trouvé
    error_vector = c_new
    print("Vecteur d'erreur:", error_vector)

    return error_vector
nearest_plane_attack(c, Bpub, n)

Vecteur d'erreur: (20, -11, 1, 69, -28, 26, -11, -68, 12, -88, 63, -31, 47, 76, 5, 35, 64, -29, 34, 74, -102, -21, 3, 140, -49, 100, 2, -13, -44, 52, -99, -16)


(20, -11, 1, 69, -28, 26, -11, -68, 12, -88, 63, -31, 47, 76, 5, 35, 64, -29, 34, 74, -102, -21, 3, 140, -49, 100, 2, -13, -44, 52, -99, -16)